# Setup

In [1]:
import numpy as np
import tensorflow as tf
import sys

# Helper Functions
- Generate Training Data

In [2]:
def generate_training_data(text, T_x=40, stride=3):
    '''
    Create a training data by scanning a window of size T_x over the text corpus, with stride 3.
    
    Arguments:
    text   -- string, corpus of Shakespearian poem
    T_x    -- integer, number of time-steps (or characters) in one training example
    stride -- integer, how much the window shift itself while scanning
    
    Returns:
    X      -- list of training examples
    Y      -- list of training labels
    '''
    
    # Initialize
    X, Y = [], []
    
    # Loop
    for i in range(0, len(text)-T_x, stride):
        X.append(text[i: (i+T_x)])
        Y.append(text[i+T_x])
    
    print('number of raw training examples: ', len(X))
    
    return X, Y

- Vectorization

In [3]:
def vectorization(X, Y, n_x, char_to_idx, T_x=40):
    '''
    Convert X and Y (lists) into arrays to be fed into RNN.
    
    Returns:
    x      -- array of shape (m, T_x, n_x)
    y      -- array of shape (m, n_x)
    '''
    
    m = len(X)
    x = np.zeros((m, T_x, n_x), dtype=np.bool)
    y = np.zeros((m, n_x), dtype=np.bool)
    
    for i, sentence in enumerate(X):
        for t, char in enumerate(sentence):
            x[i, t, char_to_idx[char]] = 1
        y[i, char_to_idx[Y[i]]] = 1
    
    print('x.shape = ', x.shape)
    print('y.shape = ', y.shape)
    
    return x, y

- Sampling

In [4]:
def sampling(preds, temperature=1.0):
    
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds - np.max(preds))
    probas = exp_preds / exp_preds.sum(axis=0)
    out = np.random.choice(range(len(chars)), p=probas.ravel())
    
    return out

- Generate Output

In [5]:
def generate_output(model):
    
    generated = ''
    
    usr_input = input('Input the beginning of your poem: ')
    
    sentence = ('{0:0>' + str(T_x) + '}').format(usr_input).lower()
    generated += usr_input
    
    sys.stdout.write('\n\nHere is your poem: \n\n')
    sys.stdout.write(usr_input)
    for i in range(400):
        
        x_pred = np.zeros((1, T_x, len(chars)))
        
        for t, char in enumerate(sentence):
            if char != '0':
                x_pred[0, t, char_to_idx[char]] = 1
                
        preds = model.predict(x_pred, verbose=0)[0]
        next_idx = sampling(preds)
        next_char = idx_to_char[next_idx]
        
        generated += next_char
        sentence = sentence[1:] + next_char
        
        sys.stdout.write(next_char)
        sys.stdout.flush()
        
        if next_char == '\n':
            continue

# Data
## Overview

In [6]:
text = open('shakespeare.txt', 'r').read().lower()
chars = sorted(list(set(text)))

text_size, corpus_size = len(text), len(chars)
print('Text length: %d | Corpus length: %d' %  (text_size, corpus_size))

Text length: 94275 | Corpus length: 38


## Preprocessing

In [7]:
# hashtable mapping characters to the indices
char_to_idx = {char:i for i,char in enumerate(chars)}

# hashtable mapping indices to characters
idx_to_char = {i:char for i, char in enumerate(chars)}

## Training Set

In [8]:
# Set time-step
T_x = 40

# Raw training set
X, Y = generate_training_data(text, T_x, stride=3)

# Vectorizing training set
x, y = vectorization(X, Y, n_x=corpus_size, char_to_idx=char_to_idx)

number of raw training examples:  31412
x.shape =  (31412, 40, 38)
y.shape =  (31412, 38)


# Model

- LSTM

In [ ]:
def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = np.random.randint(0, len(text) - T_x - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + T_x]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, T_x, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_to_idx[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sampling(preds, diversity)
            next_char = idx_to_char[next_index]

            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

In [ ]:
# build model
model = tf.keras.models.Sequential()

model.add(tf.keras.layers.LSTM(units=128, input_shape=(T_x, corpus_size)))
model.add(tf.keras.layers.Dense(corpus_size, activation='softmax'))

# set RMSprop optimizer
optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.01)

# compile model
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

# set callback
print_callback = tf.keras.callbacks.LambdaCallback(on_epoch_end=on_epoch_end)

# training
model.fit(x, y, batch_size=128, epochs=60, callbacks=[print_callback])

- Load pre-trained model

In [9]:
model_load = tf.keras.models.load_model('model_shakespeare_kiank_350_epoch.h5')

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


# Implementation

In [10]:
print_callback = tf.keras.callbacks.LambdaCallback()

model_load.fit(x, y, batch_size=128, epochs=1, callbacks=[print_callback])

31412/31412 [==============================] - 20s 639us/sample - loss: 2.5657


# Generate Text

In [11]:
generate_output(model_load)

Input the beginning of your poem: Forsooth this maketh no sense 


Here is your poem: 

Forsooth this maketh no sense but work,
and cruece not besite thou brace othing,
pant is that in buss so boy what seorts
whon the dears deseow it that hadn behind,
on tay somere kifs wising the bardth othib save,
ad, bligis thy dost, yet bewigh dieds,
and in sain to well shake you aro lays,
then my sich fortore meriroh her o'anteress,
whatie pain the indient my doth and hadist love,
so de paeray asaluss leog resaty and,
cosy s